In [35]:
import schedule
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
import numpy as np

In [150]:
current_time = datetime.now()
baseurl = 'https://www.smartshanghai.com/housing/apartments-rent'

In [279]:
response = requests.get(f'{baseurl}/1605999')

In [281]:
soup_info = BeautifulSoup(response.content, "html.parser")

In [293]:
re.findall('\d+',soup_info.find('div',class_="posted-and-views").text.split('·')[-1])[0]

'55'

In [184]:


# Function to extract listing IDs posted within the last 24 hours
def get_recent_listing_ids():
    Listing_id = []
    Price = []
    N_Bedrooms = []
    N_Bathrooms = []
    page = 0
    while True:
        params = {'page': page}
        response = requests.get(baseurl, params)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            content = soup.find_all("div", class_='cont')
            
            if not content:  # Stop if there are no more listings
                break
            
            for i in range(len(content)):
                listing_time_element = content[i].find('div', class_='address').text.strip()
                time_diff = convert_time_indicator(listing_time_element)
                
                if time_diff <= timedelta(hours=23):
                    Listing_id.append(content[i].find('div').attrs['data-listingid'])
                    Price.append(''.join(content[i].find('div', class_="price").text.strip().split()[1].split(',')))
                    info = re.findall('\d+', content[i].find('div', class_='room-type').text.strip())
                    N_Bedrooms.append(info[1])
                    N_Bathrooms.append(info[2])
                else:
                    # Stop scraping when encountering a listing older than 24 hours
                    return pd.DataFrame({'listing_id': Listing_id, 'price': Price, 'N_Bedrooms': N_Bedrooms, 'N_Bathrooms': N_Bathrooms})
                
            page += 1
        else:
            print(response.status_code)
            break

    return pd.DataFrame({'listing_id': Listing_id, 'price': Price, 'N_Bedrooms': N_Bedrooms, 'N_Bathrooms': N_Bathrooms})

# Function to convert time indicator to a timedelta
def convert_time_indicator(time_str):
    if 'hour' in time_str:
        hours = int(time_str.split()[0])
        return timedelta(hours=hours)
    elif 'day' in time_str:
        days = int(time_str.split()[0])
        return timedelta(days=days)
    return timedelta()



daily_data = get_recent_listing_ids()
daily_data = daily_data.drop_duplicates().reset_index(drop=True)

In [297]:
daily_data = daily_data[:10]

In [298]:
columns = []
values = []
amenities_data = []
views_data = []

for list_id in daily_data['listing_id']:
    response = requests.get(f'{baseurl}/{list_id}')
    
    if response.status_code == 200:
        soup_info = BeautifulSoup(response.content, "html.parser")
        
        # Extract the labels and values for the current listing ID
        labels = [label.get_text(strip=True)[:-1] for label in soup_info.find_all('div', class_='details')[0].find_all(name='label')]
        row_values = [value.get_text(strip=True) for value in soup_info.find_all('div', class_='details')[0].find_all(name='div')[:-1]]
        
        # Append unique columns to the columns list
        columns.extend(label for label in labels if label not in columns)
        
        # Append values for the current listing ID while maintaining the correct order
        row = {}
        for label, value in zip(labels, row_values):
            row[label] = value
        values.append(row)
        
        # Extract amenities information
        amenities = soup_info.find_all('div', class_='amenities')[0].find_all('li', class_=['positive', 'negative'])
        amenities_dict = {}
        
        for amenity_li in amenities:
            amenity = amenity_li.text.strip()
            amenities_dict[amenity] = 1 if 'positive' in amenity_li['class'] else 0
        
        amenities_dict['listing_id'] = list_id
        amenities_data.append(amenities_dict)
        
        # Extract views information
        views = re.findall('\d+', soup_info.find('div', class_="posted-and-views").text.split('·')[-1])[0]
        views_data.append({'listing_id': list_id, 'views': views})
        
    else:
        print(f"Failed to retrieve data for listing ID: {list_id}")

# Create DataFrames for values, amenities, and views data
values_df = pd.DataFrame(values)
amenities_df = pd.DataFrame(amenities_data)
views_df = pd.DataFrame(views_data)

# Merge the amenities DataFrame with the original DataFrame on 'listing_id'
merged_df = pd.merge(daily_data, amenities_df, on='listing_id', how='left')

# Merge the views DataFrame with the merged DataFrame on 'listing_id'
merged_df = pd.merge(merged_df, views_df, on='listing_id', how='left')

# Concatenate the merged DataFrame with the values DataFrame
final_df = pd.concat([merged_df, values_df], axis=1)
final_df.columns = final_df.columns.str.replace("\n", "").str.replace(" ", "")

In [300]:
final_df.columns

Index(['listing_id', 'price', 'N_Bedrooms', 'N_Bathrooms', 'AirFilter',
       'Balcony', 'EnglishSpeakingLandlord', 'FitnessCenters', 'FloorHeating',
       'Garden', 'HistoricBuilding', 'LandlordlivesinShanghai',
       'LargeStorageRoom', 'Oven', 'Parking', 'PetsAllowedfalse', 'Playground',
       'Pool', 'TennisCourts', 'Villa', 'Wallheating', 'WaterFilter',
       'PetsAllowedtrue', 'views', 'Type', 'AvailableFrom',
       'MinimalRentalPeriod', 'DepositRequirement', 'AdvanceRentPayment',
       'AgencyCommission', 'Rooms', 'Size', 'Floor', 'Furnished',
       'MainWindowFacing', 'District', 'Area', 'Compound', 'MetroStation'],
      dtype='object')

In [302]:

#Data cleaning
final_df.columns = final_df.columns.str.replace("\n", "").str.replace(" ", "")

# List of columns to clean
columns_to_clean = ['MinimalRentalPeriod', 'DepositRequirement', 'AdvanceRentPayment','AgencyCommission','Size']

# Define a function to clean the data in each specified column
def clean_column_data(df, column):
    df[column] = df[column].str.extract('(\d+)')

# Apply the cleaning operation to each column in the list
for column in columns_to_clean:
    clean_column_data(final_df, column)

final_df['MetroStation'] = final_df['MetroStation'].str.split('to').str[1].str.split('on ').str[0].str.strip()

# Keep only 'PetsAllowedtrue' column and rename it to 'PetsAllowed'
final_df['PetsAllowed'] = final_df['PetsAllowedtrue']

# Drop 'PetsAllowedtrue' column
final_df.drop(columns=['PetsAllowedtrue','Rooms'], inplace=True)

# Fill NaN values in the 'PetsAllowed' column with 0
final_df['PetsAllowed'] = final_df['PetsAllowed'].fillna(0)
final_df = final_df.fillna(0)

In [303]:
final_df

,listing_id,price,N_Bedrooms,N_Bathrooms,AirFilter,Balcony,EnglishSpeakingLandlord,FitnessCenters,FloorHeating,Garden,...,AgencyCommission,Size,Floor,Furnished,MainWindowFacing,District,Area,Compound,MetroStation,PetsAllowed
0,1605999,45000,4,3,0,0,0,0,1,0,...,0,220,11,Furnished,South,Huangpu,Nanjing Dong Lu,Zhongfu City III / 中福城三期,People's Square,0.0
1,1605650,7000,1,1,0,1,0,0,0,1,...,35,50,1,Furnished,South,Changning,-,Fu'an Unit / 福安小区,Jiangsu Rd,1.0
2,1606385,21500,4,2,0,0,0,0,0,0,...,0,150,27,Furnished,South,Huangpu,-,Shuijing Yuan / 水景苑,Xinzha Rd,0.0
3,1606384,5100,1,1,0,0,0,0,0,0,...,50,35,1,Furnished,South,Changning,-,Xinmei Quanyuan / 新梅泉苑,West Yan'An Rd,0.0
4,1606383,16000,2,1,1,0,0,0,0,0,...,35,120,3,Furnished,South,Jing'an,Old Xuhui,Changle Xincun / 长乐新村,South Shaanxi Rd,1.0
5,1606382,13000,1,1,1,1,0,0,0,0,...,35,70,4,Furnished,South,Jing'an,-,Xin Fu Kang Li / 新福康里,West Nanjing Rd,1.0
6,1606380,15000,2,1,1,1,0,0,0,0,...,35,110,3,Furnished,South,Jing'an,-,Xin Fu Kang Li / 新福康里,West Nanjing Rd,1.0
7,1606379,8200,1,1,0,0,0,0,0,0,...,50,60,3,Furnished,South,Putuo,-,Putuo Yi Cun / 普陀一村,Caoyang Road,0.0
8,1606378,7500,1,1,1,0,0,0,0,0,...,35,68,3,Furnished,South,Jing'an,Jing'an Temple,Yugu Village / 愚谷村,Jing'an Temple,1.0
9,1606377,15000,4,2,1,0,0,0,0,0,...,35,160,1,Furnished,South,Changning,-,Jiada Xinyuan / 佳达新苑,Jiangsu Rd,1.0


In [278]:
final_df.to_csv(f"{current_time.strftime('%Y%m%d')}.csv", index=False)

'20240814'